In [5]:
import scipy as sp
import numpy as np
import math
from mpmath import besseli
import utm

In [ ]:
samplingInterval = 60
timeWindow = 51
cutlength = 45
cutv = 4.1667
constv = 34.7/3.6

def Likelihoodww(data1,data2,cv): # calculate log-likelihood of the model
    def f(par):
        a = par[1]
        b = cv/sp.gamma(1+1/a)
        mx = par[2]
        my = par[3]
        wx = par[4]
        wy = par[5]
        L = 0
        for i in range(len(data1)):
            rr = np.sqrt((data1[i]*np.cos(data2[i]) - wx)**2 + (data1[i]*np.sin(data2[i]) - wy)**2)
            rx = (data1[i]*np.cos(data2[i])-wx)/rr
            ry = (data1[i]*np.sin(data2[i])-wy)/rr
            lp = (a-2)*math.log(rr) - (rr/b)**a + mx*rx + my*ry + math.log(a) - math.log(b) + (1-a)*math.log(b) - math.log(besseli(np.sqrt(mx**2 + my**2),0,))
            L = L+lp
        return L

def Weibull_sd(a,b): # standard deviation of Weibull distribution
    return b*np.sqrt(sp.gamma(1+2/a) - sp.gamma(1+1/a)*sp.gamma(1+1/a))

def Weibull_mean(a,b): # mean of Weibull distribution
    return b*sp.gamma(1+1/a)

def Von_Mises_sd(kappa): # standard deviation of von Mises distribution
    return 1/np.sqrt(kappa)

def readAxyGPS(filename): # read in AxyTrek GPS data (txt files)
    return read.delim(filename, sep = "\t", header = F)

def nearest(items, pivot): # find the nearest time position
    return min(items, key=lambda x: abs(x - pivot))

def timeRescale(DT,tdiff): # calculated indeces for rescaling time (DT) for regular sampling every tdiff mins
    tsel = pd.date_range(DT[0],DT[-1],freq="1T")
    return [nearest(DT,x) for x in tsel]

def spTrav(DT,lat,lon): # speed from time (DT), lat, and lon
    geod = pyproj.Geod(ellps='WGS84')
    _, _, distance = geod.inv(lon[0:-2],lat[0:-2],lon[1:-1],lat[1:-1])
    speed = (distance*10**-3)/np.array(np.diff(DT)/np.timedelta64(3600,'s')
    if threshold != 0:
        while np.nanmax(speed) > threshold:
            lat = lat[speed < threshold]
            lon = lon[speed < threshold]
            _, _, distance = geod.inv(lon[0:-2],lat[0:-2],lon[1:-1],lat[1:-1])
            speed = (distance*10**-3)/np.array(np.diff(DT)/np.timedelta64(3600,'s')
    return distance, speed

def XYfromUTM(lat,lon):
    return utm.from_latlon(lat,lon)

In [4]:
# np.arange(DT[0], DT[-1], timedelta(minutes=1)).astype(datetime)
